In [2]:
# Batch normalization
from tensorflow import keras
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, activation='elu', kernel_initializer='he_normal'),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, activation='elu', kernel_initializer='he_normal'),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(10, activation='softmax')
])

In [3]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
batch_normalization (BatchNo (None, 784)               3136      
_________________________________________________________________
dense (Dense)                (None, 300)               235500    
_________________________________________________________________
batch_normalization_1 (Batch (None, 300)               1200      
_________________________________________________________________
dense_1 (Dense)              (None, 300)               90300     
_________________________________________________________________
batch_normalization_2 (Batch (None, 300)               1200      
_________________________________________________________________
dense_2 (Dense)              (None, 10)                3

In [4]:
[(var.name, var.trainable) for var in model.layers[1].variables]

[('batch_normalization/gamma:0', True),
 ('batch_normalization/beta:0', True),
 ('batch_normalization/moving_mean:0', False),
 ('batch_normalization/moving_variance:0', False)]

In [5]:
model.layers[1].updates

[<tf.Operation 'cond/Identity' type=Identity>,
 <tf.Operation 'cond_1/Identity' type=Identity>]

In [8]:
# batch normalization before activation
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28,28]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, kernel_initializer='he_normal', use_bias=False),
    keras.layers.BatchNormalization(),
    keras.layers.Activation('elu'),
    keras.layers.Dense(100, kernel_initializer='he_normal', use_bias=False),
    keras.layers.BatchNormalization(),
    keras.layers.Activation('elu'),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(10, activation='softmax')
])

In [10]:
# gradient clipping (used in RNNs due to hard implementation of batch normalization)
optimizer = keras.optimizers.SGD(clipvalue=1.0, clipnorm=1.0)
model.compile(loss='mse', optimizer=optimizer)

In [13]:
# Transfer learning w/ Keras
import numpy as np

(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]


def split_dataset(X, y):
    y_5_or_6 = (y == 5) | (y == 6) # sandals or shirts
    y_A = y[~y_5_or_6]
    y_A[y_A > 6] -= 2 # class indices 7, 8, 9 should be moved to 5, 6, 7
    y_B = (y[y_5_or_6] == 6).astype(np.float32) # binary classification task: is it a shirt (class 6)?
    return ((X[~y_5_or_6], y_A),
            (X[y_5_or_6], y_B))

(X_train_A, y_train_A), (X_train_B, y_train_B) = split_dataset(X_train, y_train)
(X_valid_A, y_valid_A), (X_valid_B, y_valid_B) = split_dataset(X_valid, y_valid)
(X_test_A, y_test_A), (X_test_B, y_test_B) = split_dataset(X_test, y_test)
X_train_B = X_train_B[:200]
y_train_B = y_train_B[:200]

In [14]:
X_train_A.shape, X_train_B.shape, y_train_A[:10], y_train_B[:10]

((43986, 28, 28),
 (200, 28, 28),
 array([4, 0, 5, 7, 7, 7, 4, 4, 3, 4], dtype=uint8),
 array([1., 1., 0., 0., 0., 0., 1., 1., 1., 0.], dtype=float32))

In [17]:
import tensorflow as tf
from tensorflow import keras

tf.random.set_seed(42)
np.random.seed(42)

In [24]:
model_A = keras.models.Sequential()
model_A.add(keras.layers.Flatten(input_shape=[28,28]))
for n_hidden in (300, 100, 50, 50, 50):
    model_A.add(keras.layers.Dense(n_hidden, activation='selu'))
model_A.add(keras.layers.Dense(8, activation='softmax'))

In [25]:
model_A.compile(loss='sparse_categorical_crossentropy',
                optimizer=keras.optimizers.SGD(lr=1e-3),
                metrics=['accuracy'])

In [27]:
history = model_A.fit(X_train_A, y_train_A, epochs=20, validation_data=(X_valid_A, y_valid_A))

Train on 43986 samples, validate on 4014 samples
Epoch 1/20
43986/43986 [==============================] - 6s 144us/sample - loss: 0.5799 - accuracy: 0.8160 - val_loss: 0.3762 - val_accuracy: 0.8752
Epoch 2/20
43986/43986 [==============================] - 5s 119us/sample - loss: 0.3587 - accuracy: 0.8776 - val_loss: 0.3231 - val_accuracy: 0.8906
Epoch 3/20
43986/43986 [==============================] - 9s 195us/sample - loss: 0.3214 - accuracy: 0.8895 - val_loss: 0.2998 - val_accuracy: 0.8994
Epoch 4/20
43986/43986 [==============================] - 5s 119us/sample - loss: 0.3015 - accuracy: 0.8965 - val_loss: 0.2850 - val_accuracy: 0.9063
Epoch 5/20
43986/43986 [==============================] - 5s 107us/sample - loss: 0.2881 - accuracy: 0.9009 - val_loss: 0.2764 - val_accuracy: 0.9071
Epoch 6/20
43986/43986 [==============================] - 6s 134us/sample - loss: 0.2776 - accuracy: 0.9043 - val_loss: 0.2681 - val_accuracy: 0.9113
Epoch 7/20
43986/43986 [===========================

In [28]:
model_A.save('my_model_A.h5')

In [30]:
model_B = keras.models.Sequential()
model_B.add(keras.layers.Flatten(input_shape=[28,28]))
for n_hidden in (300, 100, 50, 50, 50):
    model_B.add(keras.layers.Dense(n_hidden, activation='selu'))
model_B.add(keras.layers.Dense(1, activation='sigmoid'))

In [31]:
model_B.compile(loss='binary_crossentropy', optimizer=keras.optimizers.SGD(lr=1e-3), metrics=['accuracy'])
model_B.fit(X_train_B, y_train_B, epochs=20, validation_data=(X_valid_B, y_valid_B))

Train on 200 samples, validate on 986 samples
Epoch 1/20
200/200 [==============================] - 1s 4ms/sample - loss: 0.5440 - accuracy: 0.7300 - val_loss: 0.4762 - val_accuracy: 0.8144
Epoch 2/20
200/200 [==============================] - 0s 473us/sample - loss: 0.3857 - accuracy: 0.8700 - val_loss: 0.3733 - val_accuracy: 0.8844
Epoch 3/20
200/200 [==============================] - 0s 545us/sample - loss: 0.2958 - accuracy: 0.9300 - val_loss: 0.3108 - val_accuracy: 0.9077
Epoch 4/20
200/200 [==============================] - 0s 483us/sample - loss: 0.2412 - accuracy: 0.9600 - val_loss: 0.2701 - val_accuracy: 0.9178
Epoch 5/20
200/200 [==============================] - 0s 559us/sample - loss: 0.2047 - accuracy: 0.9700 - val_loss: 0.2406 - val_accuracy: 0.9320
Epoch 6/20
200/200 [==============================] - 0s 712us/sample - loss: 0.1778 - accuracy: 0.9800 - val_loss: 0.2173 - val_accuracy: 0.9442
Epoch 7/20
200/200 [==============================] - 0s 533us/sample - loss: 0.

In [35]:
# clone in order to save the weights
model_A_clone = keras.models.clone_model(model_A)
model_A_clone.set_weights(model_A.get_weights())

model_B_on_A = keras.models.Sequential(model_A.layers[:-1])
model_B_on_A.add(keras.layers.Dense(1, activation='sigmoid'))

for layer in model_B_on_A.layers[:-1]:
    layer.trainable = False

model_B_on_A.compile(loss='binary_crossentropy',
                     optimizer=keras.optimizers.SGD(lr=1e-3),
                     metrics=['accuracy'])

# train for a few epochs to adjust the last layer (with random initializations)
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=4, validation_data=(X_valid_B, y_valid_B))

# unfreeze the layers, lower the LR and continue training
for layer in model_B_on_A.layers[:-1]:
    layer.trainable = True
    
optimizer = keras.optimizers.SGD(lr=1e-4)
model_B_on_A.compile(loss='binary_crossentropy', optimizer=keras.optimizers.SGD(lr=1e-4), metrics=['accuracy'])

Train on 200 samples, validate on 986 samples
Epoch 1/4
200/200 [==============================] - 1s 5ms/sample - loss: 0.9664 - accuracy: 0.6500 - val_loss: 0.9030 - val_accuracy: 0.6714
Epoch 2/4
200/200 [==============================] - 0s 450us/sample - loss: 0.9118 - accuracy: 0.6550 - val_loss: 0.8483 - val_accuracy: 0.6805
Epoch 3/4
200/200 [==============================] - 0s 454us/sample - loss: 0.8549 - accuracy: 0.6850 - val_loss: 0.7977 - val_accuracy: 0.6957
Epoch 4/4
200/200 [==============================] - 0s 429us/sample - loss: 0.8028 - accuracy: 0.7000 - val_loss: 0.7543 - val_accuracy: 0.7028


In [39]:
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=64, validation_data=(X_valid_B, y_valid_B))

Train on 200 samples, validate on 986 samples
Epoch 1/64
200/200 [==============================] - 0s 588us/sample - loss: 0.3622 - accuracy: 0.8200 - val_loss: 0.3541 - val_accuracy: 0.8164
Epoch 2/64
200/200 [==============================] - 0s 469us/sample - loss: 0.3505 - accuracy: 0.8200 - val_loss: 0.3423 - val_accuracy: 0.8245
Epoch 3/64
200/200 [==============================] - 0s 420us/sample - loss: 0.3382 - accuracy: 0.8200 - val_loss: 0.3329 - val_accuracy: 0.8296
Epoch 4/64
200/200 [==============================] - 0s 412us/sample - loss: 0.3284 - accuracy: 0.8250 - val_loss: 0.3243 - val_accuracy: 0.8367
Epoch 5/64
200/200 [==============================] - 0s 407us/sample - loss: 0.3195 - accuracy: 0.8250 - val_loss: 0.3154 - val_accuracy: 0.8387
Epoch 6/64
200/200 [==============================] - 0s 474us/sample - loss: 0.3102 - accuracy: 0.8250 - val_loss: 0.3074 - val_accuracy: 0.8387
Epoch 7/64
200/200 [==============================] - 0s 402us/sample - loss: 

200/200 [==============================] - 1s 3ms/sample - loss: 0.1148 - accuracy: 0.9850 - val_loss: 0.1237 - val_accuracy: 0.9838
Epoch 57/64
200/200 [==============================] - 0s 506us/sample - loss: 0.1134 - accuracy: 0.9850 - val_loss: 0.1222 - val_accuracy: 0.9848
Epoch 58/64
200/200 [==============================] - 0s 425us/sample - loss: 0.1118 - accuracy: 0.9850 - val_loss: 0.1208 - val_accuracy: 0.9848
Epoch 59/64
200/200 [==============================] - 0s 395us/sample - loss: 0.1104 - accuracy: 0.9850 - val_loss: 0.1193 - val_accuracy: 0.9848
Epoch 60/64
200/200 [==============================] - 0s 530us/sample - loss: 0.1090 - accuracy: 0.9850 - val_loss: 0.1181 - val_accuracy: 0.9868
Epoch 61/64
200/200 [==============================] - 0s 546us/sample - loss: 0.1076 - accuracy: 0.9850 - val_loss: 0.1168 - val_accuracy: 0.9878
Epoch 62/64
200/200 [==============================] - 0s 513us/sample - loss: 0.1064 - accuracy: 0.9850 - val_loss: 0.1156 - val_ac

In [37]:
model_B_on_A.evaluate(X_test_B, y_test_B)

2000/2000 [==============================] - 0s 104us/sample - loss: 0.6739 - accuracy: 0.7095


[0.6738625311851502, 0.7095]

In [42]:
from functools import partial

RegularizedDense = partial(keras.layers.Dense,activation="elu",
                                              kernel_initializer="he_normal",
                                              kernel_regularizer=keras.regularizers.l2(0.01))

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    RegularizedDense(300),
    RegularizedDense(100),
    RegularizedDense(10, activation="softmax", kernel_initializer="glorot_uniform")
])

In [43]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_6 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_25 (Dense)             (None, 300)               235500    
_________________________________________________________________
dense_26 (Dense)             (None, 100)               30100     
_________________________________________________________________
dense_27 (Dense)             (None, 10)                1010      
Total params: 266,610
Trainable params: 266,610
Non-trainable params: 0
_________________________________________________________________
